In [2]:
import json
import re

def clean_data(data):
    cleaned_data = []

    for university in data:
        cleaned_university = {
            "มหาวิทยาลัย": university["มหาวิทยาลัย"],
            "รายละเอียด": {}
        }

        for branch, programs in university["รายละเอียด"].items():
            clean_branch = re.sub(r"^\d+\.\s*", "", branch)
            cleaned_university["รายละเอียด"][clean_branch] = []

            for program in programs:
                clean_program = {}
                for key, value in program.items():
                    if key == "":
                        continue
                    elif key.startswith("รอบ"):
                        if "ไม่เปิดรับสมัครในรอบนี้" in value:
                            clean_program[key] = 0
                        else:
                            match = re.search(r"\d+", value)
                            clean_program[key] = int(match.group()) if match else 0
                    else:
                        clean_program[key] = value

                clean_program = {k: v for k, v in clean_program.items() if v != ""}
                cleaned_university["รายละเอียด"][clean_branch].append(clean_program)

        cleaned_data.append(cleaned_university)

    return cleaned_data

with open("../data/all_universities_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

cleaned_data = clean_data(data)

with open("../data/cleaned_universities_data.json", "w", encoding="utf-8") as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print("Data cleaned and saved to cleaned_data.json")


Data cleaned and saved to cleaned_data.json


#Convert to CSV

In [11]:
import json
import csv

# Load JSON data
with open("../data/cleaned_universities_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Function to replace empty values or "-" with "ไม่ระบุ"
def replace_empty_values(value):
    if value == "" or value == "-" or value == "_":
        return "ไม่ระบุ"
    return value

# Prepare CSV data
csv_data = []
csv_headers = [
    "มหาวิทยาลัย", "คณะ", "ชื่อหลักสูตร", "ชื่อหลักสูตรภาษาอังกฤษ", "ประเภทหลักสูตร", "วิทยาเขต",
    "ค่าใช้จ่าย", "อัตราการสำเร็จการศึกษา", "อัตราการได้งานทำ", "ค่ามัธยฐานเงินเดือน",
    "รอบ 1 Portfolio", "รอบ 2 Quota", "รอบ 3 Admission", "รอบ 4 Direct Admission"
]

for university in data:
    university_name = university["มหาวิทยาลัย"]
    for branch, programs in university["รายละเอียด"].items():
        for program in programs:
            row = {
                "มหาวิทยาลัย": replace_empty_values(university_name),
                "คณะ": replace_empty_values(branch),
                "ชื่อหลักสูตร": replace_empty_values(program.get("ชื่อหลักสูตร", "")),
                "ชื่อหลักสูตรภาษาอังกฤษ": replace_empty_values(program.get("ชื่อหลักสูตรภาษาอังกฤษ", "")),
                "ประเภทหลักสูตร": replace_empty_values(program.get("ประเภทหลักสูตร", "")),
                "วิทยาเขต": replace_empty_values(program.get("วิทยาเขต", "")),
                "ค่าใช้จ่าย": replace_empty_values(program.get("ค่าใช้จ่าย", "")),
                "อัตราการสำเร็จการศึกษา": replace_empty_values(program.get("อัตราการสำเร็จการศึกษา", "")),
                "อัตราการได้งานทำ": replace_empty_values(program.get("อัตราการได้งานทำ", "")),
                "ค่ามัธยฐานเงินเดือน": replace_empty_values(program.get("ค่ามัธยฐานเงินเดือน", "")),
                "รอบ 1 Portfolio": replace_empty_values(program.get("รอบ 1 Portfolio", "")),
                "รอบ 2 Quota": replace_empty_values(program.get("รอบ 2 Quota", "")),
                "รอบ 3 Admission": replace_empty_values(program.get("รอบ 3 Admission", "")),
                "รอบ 4 Direct Admission": replace_empty_values(program.get("รอบ 4 Direct Admission", ""))
            }
            csv_data.append(row)

# Save CSV data
with open("../data/cleaned_universities_data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=csv_headers)
    writer.writeheader()
    writer.writerows(csv_data)

print("Data has been converted to CSV and saved to data.csv")


Data has been converted to CSV and saved to data.csv


#Finding Lat,Lon

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

def get_lat_lon_google_maps(university_name):
    options = Options()
    # options.add_argument('--headless')  # Run Chrome in headless mode
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        driver.get("https://www.google.com/maps")
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(university_name + Keys.RETURN)
        time.sleep(5)  # Wait for the page to load and results to appear
        
        # Extract latitude and longitude from the URL
        current_url = driver.current_url
        if "/place/" in current_url:
            coords = current_url.split("@")[1].split(",")
            lat = coords[0]
            lon = coords[1]
            return lat, lon
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()
    
    return "ไม่ระบุ", "ไม่ระบุ"

def replace_empty_values(value):
    if value == "" or value == "-":
        return "ไม่ระบุ"
    return value

# Read data from input CSV file using pandas
input_file = "../data/cleaned_universities_data.csv"
output_file = "data_with_coordinates.csv"

df = pd.read_csv(input_file)
df = df.drop_duplicates(subset=['มหาวิทยาลัย'])

# Ensure column names are correctly read
df.columns = [col.strip() for col in df.columns]

# Initialize new columns for latitude and longitude
df['ละติจูด'] = ''
df['ลองจิจูด'] = ''

# Iterate through rows to get coordinates
for idx, row in df.iterrows():
    university_name = row['มหาวิทยาลัย']
    latitude, longitude = get_lat_lon_google_maps(university_name)
    df.at[idx, 'ละติจูด'] = replace_empty_values(latitude)
    df.at[idx, 'ลองจิจูด'] = replace_empty_values(longitude)

# Replace empty values or "-" with "ไม่ระบุ"
df = df.applymap(replace_empty_values)

# Save the updated dataframe to a new CSV file
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Data with coordinates has been saved to {output_file}")


Data with coordinates has been saved to data_with_coordinates.csv


/tmp/ipykernel_28715/3121733000.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(replace_empty_values)


In [12]:
import pandas

df = pandas.read_csv("data_with_coordinates.csv")

df = df[['คณะ','ละติจูด','ลองจิจูด']]
df

,คณะ,ละติจูด,ลองจิจูด
0,วิศวกรรมทั่วไป,ไม่ระบุ,ไม่ระบุ
1,วิศวกรรมคอมพิวเตอร์,ไม่ระบุ,ไม่ระบุ
2,วิศวกรรมเกษตร,16.4465919,103.5248738
3,วิศวกรรมเกษตร,16.47441,102.811746
4,วิศวกรรมคอมพิวเตอร์,ไม่ระบุ,ไม่ระบุ
5,วิศวกรรมคอมพิวเตอร์,ไม่ระบุ,ไม่ระบุ
6,วิศวกรรมการผลิต,13.8191368,100.5040095
7,วิศวกรรมทั่วไป,14.8817713,102.0104179
8,วิศวกรรมเครื่องกล,ไม่ระบุ,ไม่ระบุ
9,วิศวกรรมทั่วไป,ไม่ระบุ,ไม่ระบุ


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

def get_lat_lon_google_maps(university_name):
    options = Options()
    options.add_argument('--headless')  # Run Chrome in headless mode
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        driver.get("https://www.google.com/maps")
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(university_name + Keys.RETURN)
        time.sleep(10)  # Increase wait time for the page to load and results to appear
        
        # Extract latitude and longitude from the URL
        current_url = driver.current_url
        if "/place/" in current_url:
            coords = current_url.split("@")[1].split(",")
            lat = coords[0]
            lon = coords[1]
            return lat, lon
    except Exception as e:
        print(f"An error occurred while fetching location for {university_name}: {e}")
    finally:
        driver.quit()
    
    return "ไม่ระบุ", "ไม่ระบุ"

def replace_empty_values(value):
    if value == "" or value == "-":
        return "ไม่ระบุ"
    return value

# Read data from input CSV file using pandas
input_file = "../data/cleaned_universities_data.csv"
output_file = "data_with_coordinates_02.csv"

df = pd.read_csv(input_file)
df = df.drop_duplicates(subset=['มหาวิทยาลัย'])
# Ensure column names are correctly read
df.columns = [col.strip() for col in df.columns]

# Initialize new columns for latitude and longitude
df['ละติจูด'] = ''
df['ลองจิจูด'] = ''

# Iterate through rows to get coordinates
for idx, row in df.iterrows():
    university_name = row['มหาวิทยาลัย']
    latitude, longitude = get_lat_lon_google_maps(university_name)
    df.at[idx, 'ละติจูด'] = replace_empty_values(latitude)
    df.at[idx, 'ลองจิจูด'] = replace_empty_values(longitude)

# Replace empty values or "-" with "ไม่ระบุ"
df = df.applymap(replace_empty_values)

# Save the updated dataframe to a new CSV file
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Data with coordinates has been saved to {output_file}")


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Display all rows
pd.set_option('display.max_rows', None)

# Display all columns
pd.set_option('display.max_columns', None)

df1: pd.DataFrame = pd.read_csv('../data/cleaned_universities_data.csv')
df2: pd.DataFrame = pd.read_csv('data_with_coordinates.csv')

df2 = df2[['มหาวิทยาลัย', 'ละติจูด', 'ลองจิจูด']]

df_merged = pd.merge(df1, df2, on='มหาวิทยาลัย', how='inner')
df_merged.to_csv("cleaned_universities_data.csv")